<!--<center><h1><font color="orange">Forex Data Science</font></h1></center> -->
<img src="Images/P1.png"/>
<!--<center><h2>Predicting the Next Day Price of the EUR/USD currency pair</h2></center> -->
<hr>

<h2>Introduction:</h2>
<h4>Register Your Application:</h4>
<ol>
<li>Step 1:Go to https://developer.preprod.fusionfabric.cloud. This is the platform portal which will allow us to register an application and utilize the many APIs offered.</li>
<li>Step 2: Click the "My Dashboard" link at the the top right of the homepage. This will take you to your dashboard which holds all of your applications.</li>
<li>Step 3: Create a new application by clicking the new application button in purple at the top right.</li>
<li>Step 4: Enter in all application details (name, description, category tags, and reply url).</li>
<li>Step 5: Choose your APIs, select all Treasury and Capital markets APIs. We will be using the fxrates api.</li>
<li>Step 6: Read over and accept the terms and conditions to proceed.</li>
<li>Step 7: Go to your dashboard, select your new application and you should have your client id, access key, and reply url. We will need these to get our authentication token and access the APIs.</li>
</ol>

<h4>Save Credentials:</h4>
<p>Save your application id, consumer secret, and reply url in a csv file called credentials.csv. The file should look as follows:</p>
<p>credentials.csv<br>------------------- </p>
<p>application_id,access_key,reply_url,token_endpoint,base_url<br>your_app_id,your_consumer_secret,your_reply_url,yout_token_endpoint,your_base_url</p>

<h2>Data Science:</h2>
<h4>Imports:</h4>
<p>We need the following libraries for our project. If you are missing one of the libraries use the pip package installer by going to the terminal and typing in <b>pip install package_name</b> or enter <b>!pip install package_name</b> in a jupyter cell.</p>
<p>We will be using the following libraries:</p>
<ul>
    <li>requests</li>
    <li>re</li>
    <li>json</li>
    <li>numpy</li>
    <li>pandas</li>
    <li>datetime</li>
    <li>matplotlib</li>
    <li>seaborn</li>
    <li>pygal</li>
    <li>sklearn</li>
    <li>keras</li>
</ul>

In [2]:
#API Handling Libraries -- requests, re, json
import requests
import re
import json 

#Data Handling Libraries -- numpy, pandas, datetime
import numpy as np
import pandas as pd
import datetime
from datetime import date
from pandas_datareader import data

#Visualization Libraries -- matplotlib, seaborn, and pygal
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
import seaborn as sns
import pygal
#Some code to display the pygal charts in the Jupyter notebook
base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""
def galplot(chart):
    rendered_chart = chart.render(is_unicode=True)
    plot_html = base_html.format(rendered_chart=rendered_chart)
    display(HTML(plot_html))
display(HTML("<style>.container { height:80% !important; }</style>"))
pd.options.display.max_columns = None

#Machine Learning Libraries -- sklearn and keras
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input, LSTM, Dense

Using TensorFlow backend.


<h4>Authentication Token:</h4>
<p>To get data from a given API we first need to get our authentication token. To do so, we create the following class which will take in our credentials.csv file and grab our authentication token.</p>

In [3]:
class authenticate: 
    '''
    credentials_file: A csv that stores the client id, client secret, and reply url
    '''
    def __init__(self,config_file): 
        self.df = pd.read_csv(config_file)
        self.client_id = self.df['application_id'].values[0]
        self.client_secret = self.df['access_key'].values[0]
        self.reply_url = self.df['reply_url'].values[0]
        self.token_endpoint = self.df['token_endpoint'].values[0]
        self.base_url = self.df['base_url'].values[0]
        
    def get_token(self): 
        headers = {
        }
        data = {
            'grant_type'     : 'client_credentials',
            'client_id'      : self.client_id,
            'client_secret'  : self.client_secret
        }
        response = requests.request('POST', self.token_endpoint, data=data, headers=headers)
        cleaned_response = response.text.replace(':null', ':"null"')
        response_dict = eval(cleaned_response)
        token = response_dict['access_token']
        return response, token
    
a = authenticate('config.csv')
response,token = a.get_token()
print(response.status_code)

KeyError: 'access_token'

In [ ]:
token

<h4>Get Foreign Exchange Data:</h4>
<p>This class will take in our access token, a POST or GET method, and endpoint we want to access, a payload, an option for additional headers, and any parameters we want to specify in the request. We will use this to get FX Rates data for the Euro against other currencies for a particular day.</p>

In [5]:
class tcm_api: 
    def __init__(self,config_file,token,method,endpoint,payload,additional_headers={},params = None): 
        self.df = pd.read_csv(config_file)
        self.base_url = self.df['base_url'].values[0]
        print(self.base_url)
        self.token = token 
        self.endpoint_type = method
        self.endpoint = '/fxrate/v1/'+endpoint 
        self.additional_headers = additional_headers
        self.params = params
    
    def connect_endpoint(self):
        self.url = self.base_url + self.endpoint
        print(self.url)
        headers = {
            'Authorization': 'Bearer ' + self.token,
            'Content-Type' : 'application/json',
            }
        for key in self.additional_headers.keys():
            headers[key] = self.additional_headers[key]
        stringified_payload = json.dumps(payload)
        response = requests.request(self.endpoint_type, self.url, params=self.params, data=stringified_payload,headers=headers)
        return response

#Example GET Request for Forex Data on 2019-02-01
method = 'GET'
endpoint = '2019-02-01'
payload = ''
additional_headers = {}
params = {}
config_file = 'config.csv'
a = authenticate(config_file)
#response,token = a.get_token()
tcm = tcm_api(config_file,token,method,endpoint,payload,additional_headers,params)
response = tcm.connect_endpoint()
print("Response: ",response.text)

https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-02-01
Response:  {"base":"EUR","rates":{"BGN":1.9558,"NZD":1.6567,"ILS":4.1649,"RUB":75.1176,"CAD":1.5075,"USD":1.1471,"PHP":59.916,"CHF":1.1396,"ZAR":15.2861,"AUD":1.5789,"JPY":124.91,"TRY":5.9884,"HKD":9.0012,"MYR":4.6839,"THB":35.893,"HRK":7.415,"NOK":9.6685,"IDR":16009.5,"DKK":7.4654,"CZK":25.695,"HUF":317.63,"GBP":0.87888,"MXN":21.9455,"KRW":1281.33,"ISK":137.8,"SGD":1.5472,"BRL":4.211,"PLN":4.2739,"INR":81.771,"RON":4.7557,"CNY":7.7262,"SEK":10.3878},"date":"2019-02-01"}


<h4>Generate Dates to call the API with:</h4>
<p>Let's get 20 years of historical FX rates. To do so, first we need to generate a list of dates and attach each of these dates as an endpoint in the API.</p>

In [6]:
def generate_dates(numdays):
    #Get todays date and all number of numdays previous and convert them to a string in Year-Month-Day Format
    base = datetime.datetime.today()
    date_list = [(base - datetime.timedelta(days=x)).strftime("%Y-%m-%d") for x in range(0, numdays)]
    return date_list

years = 20
dates = generate_dates(numdays=365*years)
print("Dates Sample: ",dates[0:10])

Dates Sample:  ['2019-05-09', '2019-05-08', '2019-05-07', '2019-05-06', '2019-05-05', '2019-05-04', '2019-05-03', '2019-05-02', '2019-05-01', '2019-04-30']


<h4>Get 20 years of EUR/USD data:</h4>
<p>The following code iterates over all dates, attaches them as an endpoint, returns the fx rates for that day, extracts the EUR/USD rate, determines if the date is a weekend or weekday, and adds the date to a list. We then convert the list data to a dataframe.</p>

In [7]:
#Set API Request parameters
method = 'GET'
payload = ''
config_file = 'config.csv'
additional_headers = {}
params = {}
#Iterate over all dates to get fx rate for that day
currencies_data = []
for idx,date in enumerate(dates): 
    endpoint = date 
    #Re-Authenticate every 50 requests
    if idx % 50 == 0: 
        a = authenticate(config_file)
        response,token = a.get_token()
    #Make the request
    tcm = tcm_api(config_file,token,method,endpoint,payload,additional_headers,params)
    response = tcm.connect_endpoint()
    #Extract the EUR/USD rate and add it to list as [ [rate,weekday/Weekend,date]]
    dta = eval(response.text)
    usd = dta["rates"]["USD"]
    ls = [usd]
    if dta["date"] != date: 
        ls.append("Weekend")
    else: 
        ls.append("Weekday")
    ls.append(date)
    currencies_data.append(ls)
#Create DataFrame from list of list data
df = pd.DataFrame(currencies_data)
df = df.rename(columns={0:'EUR/USD',1:'PartOfWeek',2:'Date'}) 
#Save DataFrame to CSV for later use
df.to_csv('EUR_USD_20_years.csv',index=None)

https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-05-09
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-05-08
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-05-07
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-05-06
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-05-05
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-05-04
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-05-03
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-05-02
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-05-01
https://ap

https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-02-23
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-02-22
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-02-21
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-02-20
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-02-19
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-02-18
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-02-17
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-02-16
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2019-02-15
https://ap

https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-12-10
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-12-09
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-12-08
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-12-07
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-12-06
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-12-05
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-12-04
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-12-03
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-12-02
https://ap

https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-09-26
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-09-25
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-09-24
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-09-23
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-09-22
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-09-21
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-09-20
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-09-19
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-09-18
https://ap

https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-07-13
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-07-12
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-07-11
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-07-10
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-07-09
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-07-08
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-07-07
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-07-06
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2018-07-05
https://ap

...

https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-07-07
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-07-06
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-07-05
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-07-04
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-07-03
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-07-02
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-07-01
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-06-30
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-06-29
https://ap

https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-04-23
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-04-22
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-04-21
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-04-20
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-04-19
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-04-18
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-04-17
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-04-16
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-04-15
https://ap

https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-02-07
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-02-06
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-02-05
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-02-04
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-02-03
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-02-02
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-02-01
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-01-31
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2002-01-30
https://ap

https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2001-11-24
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2001-11-23
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2001-11-22
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2001-11-21
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2001-11-20
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2001-11-19
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2001-11-18
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2001-11-17
https://api.preprod.fusionfabric.cloud/fxrate/v1/
https://api.preprod.fusionfabric.cloud/fxrate/v1/2001-11-16
https://ap

KeyError: 'access_token'

<h4>Load Saved Data:</h4>
<p>The following line of code will load in the csv data so that we do not need to make the API call again.</p>

In [ ]:
df = pd.read_csv('EUR_USD_20_years.csv')

<h4>Reorder Data:</h4>
<p>The data is currently from most recent date to least recent data. Since this is time series data we need to reorder it from oldest to newest.</p>

In [ ]:
#Reorder the Time Series Data (older dates --> newer dates)
df_clean = df.iloc[::-1]

<h4>Data Sample:</h4>
<p>We now have our data! Let's take a look at the first few rows:</p>

In [ ]:
df_clean.head(20)

<h4>Generate Features:</h4>
<p>Let's generate some features for our machine learning model. We will create some features based on dates as well as some technical indicators.</p>

In [ ]:
def add_datepart(df, fldname, drop=True):
    ''' 
    A function to calculate some date features
    '''
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)

#Generate Date Features        
add_datepart(df_clean, 'Date')

#Generate Technical indicators (Simple/Exponential Moving Averages)
df_clean['sma5'] = df_clean['EUR/USD'].rolling(5).mean()
df_clean['sma2'] = df_clean['EUR/USD'].rolling(2).mean()
df_clean['sma3'] = df_clean['EUR/USD'].rolling(3).mean()
df_clean['ema5'] = df_clean['EUR/USD'].ewm(span=5,min_periods=0,adjust=False,ignore_na=False).mean()
df_clean['ema2'] = df_clean['EUR/USD'].ewm(span=2,min_periods=0,adjust=False,ignore_na=False).mean()
df_clean['ema3'] = df_clean['EUR/USD'].ewm(span=3,min_periods=0,adjust=False,ignore_na=False).mean()

#Remove first 5 data points since will contain NaN from SMA5/EMA5
df_clean = df_clean.iloc[5:]
keep_cols = list(df_clean.columns.values)
keep_cols.remove('PartOfWeek')
df_feat = df_clean[keep_cols]

<h4>View Data:</h4>
<p>Let's take a look at some of the features we generated:</p>

In [ ]:
df_clean.head()

<h4>Feature Analysis:</h4>
<p>Let's examine the features we have created. First, let's take a look at the pearson correlation of our features. A Pearson correlation is a number between -1 and 1 that indicates the extent to which two variables are linearly related where 1 is total positive linear correlation, 0 is no linear correlation, and −1 is total negative linear correlation.</p>

In [ ]:
colormap = plt.cm.inferno
plt.figure(figsize=(15,15))
plt.title('Pearson correlation of features', y=1.05, size=15)
sns.heatmap(df_clean.corr(), linewidths=0.1, vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)
plt.show()

<h4>Create Input and Target Data:</h4>
<p>We create a shifted Forward column which contains the future price we can use for regression and a Direction column which is binary for price being higher or lower in the future that we can use for classification. We will use the Forward column and solve a regression problem in this notebook.</p>

In [ ]:
#Set number of days to predict ahead and an option to scale our data
future_prediction_days = 1
scale_data = True

#Create Forward column that has the shifted price
df_feat['Forward'] = df_feat['EUR/USD'].shift(-1*future_prediction_days)
df_feat = df_feat[:-1*future_prediction_days]

#Create Direction column to get binary shifted price direction
def get_direction(row1,row2): 
    if row2 >= row1: 
        return 1 
    else: 
        return 0
df_feat['Direction'] = df_feat[['EUR/USD','Forward']].apply(lambda i: get_direction(i[0],i[1]),axis=1)
target_index = df_feat.columns.tolist().index('Forward')
dataset = df_feat.values.astype('float32')

#Option to scale data 
if scale_data: 
    # Scale the data between 0 and 1 
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    
    y_scaler = MinMaxScaler(feature_range=(0, 1))
    t_y = df_feat['EUR/USD'].values.astype('float32')
    t_y = np.reshape(t_y, (-1, 1))
    y_scaler = y_scaler.fit(t_y)

#Separate the Input from the Target
X = dataset[:,:-2]
y = dataset[:,target_index]

<h4>Feature Analysis against Target:</h4>
<p>Now that we have our target variable we can se the correlation with our features against this specific Forward column.</p>

In [ ]:
plt.figure(figsize=(15,5))
corr = df_feat.iloc[:,:-1].corr()
sns.heatmap(corr[corr.index == 'Forward'], linewidths=0.1, vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True);

<h4>Feature Importance:</h4>
<p>Let's build a random forest and examine the feature importance before we build our actual model for prediction of the next day EUR/USD value.</p>

In [ ]:
#Build and Train a Random Forest with 100 estimators
forest = RandomForestRegressor(n_estimators = 100)
forest = forest.fit(X, y)

In [ ]:
#Extract the feature column names and feature importances for the columns
column_list = df_feat.columns.values.tolist()
importances = forest.feature_importances_

#Zip the column names and importances togther and then sort them 
col_imp = list(zip(column_list,importances))
sorted_col_imp = sorted(col_imp,key=lambda i: i[1],reverse=True)

#Create a pygal bar chart to plot the feature importances
bc = pygal.Bar(width=500, height=300, explicit_size=True)
bc.title = 'Forex Feature Importances'
for i in range(len(importances)):
    bc.add(sorted_col_imp[i][0], sorted_col_imp[i][1])
galplot(bc)

<h4>Train/Test Split:</h4>
<p>Since we have time series data, we will train on the first 80% of the data and test on the last 20% of the data.</p>

In [ ]:
#Set the percentage for training data
pct_train = .8

#Get the X,Y train/test sets
X_train = X[0:int(len(X)*pct_train)]
y_train = y[0:int(len(y)*pct_train)]
X_test = X[int(len(X)*pct_train):]
y_test = y[int(len(y)*pct_train):]

#Reshape our data into time series for an LSTM model 
X_train = X_train.reshape((X_train.shape[0],1,20))
X_test = X_test.reshape((X_test.shape[0],1,20))

<h4>Create Model:</h4>
<p>We will create a deep learning model that has the following architecture:</p>
<ul>
    <li>LSTM</li>
    <li>LSTM</li>
    <li>LSTM</li>
    <li>Dropout</li>
    <li>LSTM</li>
    <li>Dense</li>
    <li>Dense</li>
</ul>
<p>We will also use mean squared error as our loss function and the adam optimizer.</p>

In [ ]:
model = Sequential()
model.add(LSTM(20, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(20, return_sequences=True))
model.add(LSTM(10, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(4, return_sequences=False))
model.add(Dense(4, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='relu'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
print(model.summary())

<h4>Train Neural Network:</h4>
<p>We will train the network for 200 epochs and use a batch size of 500.</p>

In [ ]:
num_epochs = 200
model.fit(X_train, y_train, epochs=num_epochs, batch_size=256)

<h4>Train Set Performance:</h4>

In [ ]:
#Get Training MSE
mse = model.history.history['mean_squared_error']

#Build Line chart to show performance
line_chart = pygal.StackedLine(fill=True,width=500, height=300, explicit_size=True,
                              x_labels_major_every=10, show_minor_x_labels=False, x_label_rotation=45)
line_chart.title = 'Training Error'
line_chart.x_labels = [str(i) for i in range(num_epochs)]
line_chart.xlabel = 'Epoch'
line_chart.add('MSE', mse)
galplot(line_chart)

<h4>Test Set Performance:</h4>

In [ ]:
#Obtain Predictions on our Test Set
preds = model.predict(X_test)

#Get Evaluation Metrics -- MSE, MAE, R2 Score, and Explained Variance Score
print("Test MSE:",mean_squared_error(y_true=y_test,y_pred=preds))
print("Test MAE:  ",mean_absolute_error(y_true=y_test,y_pred=preds))
print("Test R2 Score: ",r2_score(y_pred=preds,y_true=y_test))
print("Explained Variance Score: ",explained_variance_score(y_pred=preds,y_true=y_test))

#Convert Preds and Actual Values on Test Set to Flat List for Plotting
preds_flat = preds.flatten().tolist()
y_test_flat = y_test.flatten().tolist()

#Plot Predictions vs Actual
line_chart = pygal.Line(width=500, height=300, explicit_size=True,x_labels_major_every=150, show_minor_x_labels=False, x_label_rotation=45)
line_chart.title = 'Test Predictions vs Actual'
line_chart.x_labels = df['Date'][::-1].iloc[int(0.8*len(df)):].tolist()
line_chart.add('Preds', preds_flat)
line_chart.add('Actual',  y_test_flat)
galplot(line_chart)

<h4>Save Model:</h4>
<p>We save the model, which we can turn into an API or embed in our applications.</p>

In [ ]:
model.save('EUR_USD_1_Day_Forecast_3.h5')